In [11]:
pip install tensorflow


[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, GlobalAveragePooling2D, Dense, Add, Activation

In [28]:
# Fungsi untuk membangun blok konvolusi ResNet
def conv_block(x, filters, kernel_size, strides):
    x = Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

In [36]:
# Fungsi untuk membangun blok ResNet
def resnet_block(x, filters, num_blocks, stride):
    shortcut = x
    
    for i in range(num_blocks):
        if i == 0:
            x = conv_block(x, filters, kernel_size=(3, 3), strides=stride)
        else:
            x = conv_block(x, filters, kernel_size=(3, 3), strides=1)
        
    # Skip connection
    if shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, (1, 1), strides=stride, padding='same')(shortcut)
    
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

In [37]:
# Model ResNet-50
def ResNet50(input_shape=(224, 224, 3), num_classes=1000):
    input_tensor = Input(shape=input_shape)
    
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    
    x = resnet_block(x, 64, 3, stride=1)
    x = resnet_block(x, 128, 4, stride=2)
    x = resnet_block(x, 256, 6, stride=2)
    x = resnet_block(x, 512, 3, stride=2)
    
    x = GlobalAveragePooling2D()(x)
    x = Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.models.Model(inputs=input_tensor, outputs=x, name='resnet50')
    return model

In [38]:
# Membuat model ResNet-50
model = ResNet50()
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_10 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_74 (Conv2D)          (None, 112, 112, 64)         9472      ['input_10[0][0]']            
                                                                                                  
 batch_normalization_71 (Ba  (None, 112, 112, 64)         256       ['conv2d_74[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_27 (Activation)  (None, 112, 112, 64)         0         ['batch_normalization_7